## Alocação de unidades de atendimento móvel do CBMERJ de Campos dos Goytacazes - RJ

In [1]:
# Library
import pathlib
import os
import sys

from dotenv import load_dotenv
import numpy as np
import pandas as pd
from pulp import *
from utils.process import stander_text
# my packges
sys.path.insert(0, str(pathlib.Path().absolute().parent))

import process.allocation as allocation
from process.geocode import *

load_dotenv()


True

**Importar dados**

In [5]:
df_matrix= pd.read_excel('../data/matriz_tempo.xlsx', index_col=0)

In [7]:
df_matrix

,5 GBM,DBM 1/5 GUARUS,HOSP. FERREIRA M,HOSP. G. GUARÚS,HOSP. SÃO JOSÉ,HOSP. TRAVESSÃO,PS GUARUS,SANDU,UBS Sartunino Braga,UBS TOCOS,UPA,UPH F. SÃO THOMÉ,UPH MORRO DO COCO,UPH ST EDUARDO
Clientes,,,,,,,,,,,,,,
Alphaville,1132,1997,1449,1345,438,2579,1450,972,1610,1348,1706,534,4157,5609
Baixa Grande,3004,3869,3321,3217,2098,4451,3322,2844,655,2062,3578,1765,6029,7481
Boa Vista,3536,4401,3852,3749,2629,4982,3854,3375,1186,2432,4109,2297,6561,8012
Caboio,2808,3673,3124,3021,1901,4254,3126,2647,1025,1404,3381,1568,5833,7284
Caju,508,768,150,591,1623,1331,368,811,2794,2422,412,1718,2909,4361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vila Industrial,982,223,909,800,2215,1050,826,1323,3386,3124,521,2311,2629,4080
Vila MANHÃES,1077,1942,1394,1290,885,2524,1455,999,1524,1571,1650,980,4102,5554
Vila Nova de Campos,2751,2197,2625,2570,3984,1425,2596,3093,5155,4893,2238,4080,1052,2580


# Modelo matemático

In [8]:
# DEFININDO VARIÁVEIS
location = df_matrix.columns.values
clients = df_matrix.index.values 
w = [1 for w in clients]
m = df_matrix.shape[0] # Nº de clientes
n = df_matrix.shape[1] # Nº de potenciais facilidades
p = 9 # Nº de facilidades   


### Cenários

In [9]:
scenario = {
    'sem_peso':[1 for x in range(m)],
    'ponderado_geral':[]
}

In [10]:
p_median = allocation.Allocation(m, n, p, df_matrix)
p_median.solver()
res = p_median.get_result()



1
Optimal objective value: 110595.0
Facility 1 is located. DBM 1/5 GUARUS
- Customer 12 is served. Codim
- Customer 22 is served. Estrada de Brejo Grande
- Customer 49 is served. Novo Eldorado - Parque CEASA
- Customer 51 is served. Parque Aeroporto
- Customer 74 is served. Parque Eldorado
- Customer 102 is served. Parque Santa Rosa
- Customer 105 is served. Parque Santos Dumont
- Customer 110 is served. Parque Sao Silvestre
- Customer 126 is served. Parque palmares
- Customer 148 is served. São Sebastião
- Customer 151 is served. Terra Prometida
- Customer 160 is served. Vila Industrial
Facility 2 is located. HOSP. FERREIRA M
- Customer 4 is served. Caju
- Customer 11 is served. Chácara dos Sonhos - Morro Grande
- Customer 18 is served. Dores De Macabu
- Customer 29 is served. Guriri - Dores de Macabu
- Customer 32 is served. Ibitioca
- Customer 36 is served. Jardim Maria de Queiroz
- Customer 38 is served. Lagoa De Cima
- Customer 42 is served. Mato Escuro
- Customer 45 is served. Mo

In [13]:
facilities, times, df_matrix = res

In [16]:
facilities
df_matrix

,5 GBM,DBM 1/5 GUARUS,HOSP. FERREIRA M,HOSP. G. GUARÚS,HOSP. SÃO JOSÉ,HOSP. TRAVESSÃO,PS GUARUS,SANDU,UBS Sartunino Braga,UBS TOCOS,UPA,UPH F. SÃO THOMÉ,UPH MORRO DO COCO,UPH ST EDUARDO
Clientes,,,,,,,,,,,,,,
Alphaville,0,0,0,0,0,0,0,0,0,0,0,534,0,0
Baixa Grande,0,0,0,0,0,0,0,0,655,0,0,0,0,0
Boa Vista,0,0,0,0,0,0,0,0,1186,0,0,0,0,0
Caboio,0,0,0,0,0,0,0,0,1025,0,0,0,0,0
Caju,0,0,150,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vila Industrial,0,223,0,0,0,0,0,0,0,0,0,0,0,0
Vila MANHÃES,0,0,0,0,0,0,0,0,0,0,0,980,0,0
Vila Nova de Campos,0,0,0,0,0,1425,0,0,0,0,0,0,0,0
